In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
import xgboost as xgb
from collections import Counter
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_recall_curve, auc
import numpy as np

# Load the dataset
file_path = 'Resources/US_Accidents_2022_Phoenix_Metro2.csv'
df = pd.read_csv(file_path, low_memory=False)

# Drop columns with too many missing values
df.drop(columns=['End_Lat', 'End_Lng'], inplace=True)

# Handle missing values in other columns
df.dropna(inplace=True)

# Convert date columns to datetime and then to numerical values (e.g., ordinal)
df['Accident_Date'] = pd.to_datetime(df['Accident_Date']).map(pd.Timestamp.toordinal)

# Combine classes into binary categories
df['binary_target'] = df['Severity'].apply(lambda x: 0 if x in [1, 2] else 1)

# Create labels and features
y = df['binary_target']
X = df.drop(columns=['Severity', 'binary_target'])

# Identify non-numeric columns
non_numeric_columns = X.select_dtypes(include=['object']).columns

# One-hot encode categorical columns
X = pd.get_dummies(X, columns=non_numeric_columns, drop_first=True)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

# Instantiate and fit the Logistic Regression model
clf = LogisticRegression(random_state=1, max_iter=1000)
clf.fit(X_train, y_train)

# Make predictions
y_pred = clf.predict(X_test)

# Evaluate the model
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

# Apply SMOTE to balance the classes
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# Train the logistic regression model with class weights
model = LogisticRegression(max_iter=1000, class_weight='balanced')
model.fit(X_train_smote, y_train_smote)

# Predict on the test set
y_pred = model.predict(X_test)

# Evaluate the model
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

# Feature selection using RandomForest
selector = SelectFromModel(RandomForestClassifier(n_estimators=100, random_state=42))
selector.fit(X_train_smote, y_train_smote)
X_train_smote_selected = selector.transform(X_train_smote)
X_test_selected = selector.transform(X_test)

# Train the RandomForest classifier
model = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')
model.fit(X_train_smote_selected, y_train_smote)

# Predict on the test set
y_pred = model.predict(X_test_selected)

# Evaluate the model
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

# Train the XGBoost classifier
model = xgb.XGBClassifier(n_estimators=100, random_state=42, scale_pos_weight=len(y_train_smote)/sum(y_train_smote))
model.fit(X_train_smote_selected, y_train_smote)

# Predict on the test set
y_pred = model.predict(X_test_selected)

# Evaluate the model
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

# Calculate the scale_pos_weight parameter
counter = Counter(y_train_smote)
scale_pos_weight = counter[0] / counter[1]

# Train the XGBoost classifier with adjusted class weights
model = xgb.XGBClassifier(n_estimators=100, random_state=42, scale_pos_weight=scale_pos_weight)
model.fit(X_train_smote_selected, y_train_smote)

# Predict on the test set
y_pred = model.predict(X_test_selected)

# Evaluate the model
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

# Define the parameter grid
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 4, 5],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1.0]
}

# Perform grid search
grid_search = GridSearchCV(xgb.XGBClassifier(random_state=42, scale_pos_weight=scale_pos_weight), param_grid, cv=5, scoring='f1')
grid_search.fit(X_train_smote_selected, y_train_smote)

# Get the best model from grid search
best_model = grid_search.best_estimator_

# Predict on the test set
y_pred = best_model.predict(X_test_selected)

# Evaluate the best model
print("Best Parameters:\n", grid_search.best_params_)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

# Predict probabilities
y_pred_prob = best_model.predict_proba(X_test_selected)[:, 1]

# Calculate precision-recall curve
precision, recall, thresholds = precision_recall_curve(y_test, y_pred_prob)
f1_scores = 2 * (precision * recall) / (precision + recall)

# Find the best threshold
best_threshold = thresholds[np.argmax(f1_scores)]

# Adjust predictions based on the best threshold
y_pred_adjusted = (y_pred_prob >= best_threshold).astype(int)

# Evaluate the adjusted predictions
print("Best threshold:", best_threshold)
print(confusion_matrix(y_test, y_pred_adjusted))
print(classification_report(y_test, y_pred_adjusted))


[[176   0]
 [ 13   0]]
              precision    recall  f1-score   support

           0       0.93      1.00      0.96       176
           1       0.00      0.00      0.00        13

    accuracy                           0.93       189
   macro avg       0.47      0.50      0.48       189
weighted avg       0.87      0.93      0.90       189

[[168   8]
 [  8   5]]
              precision    recall  f1-score   support

           0       0.95      0.95      0.95       176
           1       0.38      0.38      0.38        13

    accuracy                           0.92       189
   macro avg       0.67      0.67      0.67       189
weighted avg       0.92      0.92      0.92       189

[[176   0]
 [ 13   0]]
              precision    recall  f1-score   support

           0       0.93      1.00      0.96       176
           1       0.00      0.00      0.00        13

    accuracy                           0.93       189
   macro avg       0.47      0.50      0.48       189
weigh